In [1]:
import requests

In [2]:
import json

In [3]:
from main import flatten, fix_values
import os

In [4]:
proj_endpoint = 'https://api.optimizely.com/v2/projects'
exp_endpoint = 'https://api.optimizely.com/v2/experiments'

In [5]:
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

In [6]:
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [7]:
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    return j_proj

In [ ]:
str(datetime.now())

In [8]:
from datetime import datetime

In [9]:
def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
    # get all experiments from one project
    j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

    # loop for all experiments in this project
    experiment_id_list = []
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = project_id
        exp['upload_ts'] = str(datetime.now())
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])
        
    return upload_exp_list, experiment_id_list

In [10]:
project_id_list = []

In [11]:
for p in generate_projects(proj_endpoint, headers):
    project_id_list.append(p['id'])

In [12]:
project_id_list

[9965963792, 10347872424, 10427612860, 10429921250, 10849384554]

In [13]:
params = (
    ('project_id', 9965963792),
    ('per_page', 100),
) 

In [14]:
exp_list, exp_id_list = generate_experiments(9965963792, exp_endpoint, headers, params)

In [15]:
for exp in exp_list:
    if exp['created'] >= '2018-10-23T17:48:46.952180Z':
        print(exp['id'])

11938601024
11966791385
11992110033


In [16]:
type(exp_list[0]['created'])

str

In [17]:
exp_list[0]['created'] <= str(datetime.now())

True

In [18]:
exp_list

[{'allocation_policy': 'manual',
  'audience_conditions': '["and", {"audience_id": 10769441047}]',
  'campaign_id': 10755791168,
  'created': '2018-05-23T17:48:35.952180Z',
  'description': 'Testing alternate experience for FT sign up form',
  'earliest': '2018-05-25T17:32:25.742000Z',
  'holdback': 0,
  'id': 10756790433,
  'is_classic': False,
  'last_modified': '2018-06-03T04:59:26.187040Z',
  'latest': '2018-06-03T04:59:28.088920Z',
  'metrics_aggregator': 'unique',
  'metrics_event_id': 10769480069,
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'name': 'FT Signup Variations',
  'project_id': 9965963792,
  'status': 'paused',
  'type': 'a/b',
  'upload_ts': '2019-02-14 13:03:13.944621',
  'url_targeting_activation_type': 'url_changed',
  'url_targeting_conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/setup"}, {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/signup"}]]

In [19]:
import popelines

In [20]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

In [21]:
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)


# test specific experiment

In [22]:
result_endpoint = 'https://api.optimizely.com/v2/experiments/11039523400/timeseries'
# result_endpoint = 'https://api.optimizely.com/v2/experiments/10756790433/timeseries'
response_ts = requests.get(result_endpoint, headers=headers)

In [26]:
j_ts = json.loads(response_ts.text)

In [27]:
new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

In [42]:
new_j_ts

{'confidence_threshold': 0.9,
 'end_time': '2018-07-08T18:25:00.000000Z',
 'experiment_id': 11039523400,
 'metrics': [{'aggregator': 'unique',
   'event_id': 10757331770,
   'name': 'Create new account button',
   'results': [{'is_baseline': True,
     'level': 'variation',
     'name': 'Label says "Email"',
     'results_id': '11059362372',
     'timeseries': [{'samples': 0,
       'time': '2018-07-07T22:25:27.000000Z',
       'value': 0,
       'variance': 0.0},
      {'samples': 0,
       'time': '2018-07-07T22:37:26.730000Z',
       'value': 0,
       'variance': 0.0},
      {'samples': 0,
       'time': '2018-07-07T22:49:26.460000Z',
       'value': 0,
       'variance': 0.0},
      {'rate': 0.0,
       'samples': 1,
       'time': '2018-07-07T23:01:26.190000Z',
       'value': 0,
       'variance': 0.0},
      {'rate': 0.0,
       'samples': 1,
       'time': '2018-07-07T23:13:25.920000Z',
       'value': 0,
       'variance': 0.0},
      {'rate': 0.0,
       'samples': 0,
      

In [23]:
if response_ts.text == '':
    j_ts = {'experiment_id': 10815382176}
    new_j_ts = j_ts
    new_j_ts['upload_ts'] = str(datetime.now())

else:
    j_ts = json.loads(response_ts.text)

    new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

    flattened_metrics = []
    for metric in new_j_ts['metrics']:
        for result in metric['results']:
            for timeseries in result['timeseries']:
                new_dict = {}
                new_dict['aggregator'] = metric['aggregator']
                new_dict['event_id'] = metric['event_id']
                new_dict['name'] = metric['name']
                new_dict['scope'] = metric['scope']
                new_dict['winning_direction'] = metric['winning_direction']

                new_dict['result_is_baseline'] = result['is_baseline']
                new_dict['result_level'] = result['level']
                new_dict['result_name'] = result['name']
                new_dict['result_variation_id'] = result['variation_id']
                new_dict['result_id'] = result['results_id']

                if 'rate' in timeseries.keys():
                    new_dict['time_series_rate'] = timeseries['rate']
                else:
                    pass

                new_dict['time_series_time'] = timeseries['time']
                new_dict['time_series_value'] = timeseries['value']
                new_dict['time_series_samples'] = timeseries['samples']
                new_dict['time_series_variance'] = timeseries['variance']

                flattened_metrics.append(new_dict)
    new_j_ts['metrics'] = flattened_metrics
    new_j_ts['upload_ts'] = str(datetime.now())

KeyError: 'event_id'

In [ ]:
new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

In [ ]:
for metric in new_j_ts['metrics']:
    print(metric.keys())